In [1]:
import tensorflow  as tf

In [2]:
tf.__version__

'2.3.1'

In [3]:
v = tf.Variable(0.0)

In [4]:
(v + 1).numpy()

1.0

In [5]:
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [6]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [7]:
v.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=6.0>

In [8]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>

In [9]:
v.read_value()

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [10]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as t:
    loss = w * w

In [11]:
grad = t.gradient(loss, w)

In [12]:
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [13]:
w = tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(w)
    loss = w * w

In [14]:
dloss_dw = t.gradient(loss, w)

In [15]:
dloss_dw

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [16]:
w = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w * w
    z = y * y

In [17]:
dy_dw = t.gradient(y, w)

In [18]:
dy_dw

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [19]:
dz_dw = t.gradient(z, w)

In [20]:
dz_dw

<tf.Tensor: shape=(), dtype=float32, numpy=108.0>

In [21]:
(train_image, train_labels), (test_image, test_labels) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [22]:
train_image.shape

(60000, 28, 28)

In [23]:
train_image = tf.expand_dims(train_image, -1)

In [24]:
test_image = tf.expand_dims(test_image, -1)

In [25]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [26]:
train_image = tf.cast(train_image/255, tf.float32)

In [27]:
test_image = tf.cast(test_image/255, tf.float32)

In [28]:
train_labels = tf.cast(train_labels, tf.int64)

In [29]:
test_labels = tf.cast(test_labels, tf.int64)

In [30]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_labels))

In [31]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_labels))

In [32]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [33]:
dataset = dataset.shuffle(10000).batch(32)

In [34]:
test_dataset = test_dataset.batch(32)

In [35]:
dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [36]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(32, [3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)
])

In [37]:
optimizer = tf.keras.optimizers.Adam()

In [38]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [39]:
features, labels = next(iter(dataset))

In [40]:
features.shape

TensorShape([32, 28, 28, 1])

In [41]:
labels.shape

TensorShape([32])

In [42]:
predictions = model(features)

In [43]:
predictions.shape

TensorShape([32, 10])

In [44]:
tf.argmax(predictions, axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 9, 9, 1, 1, 1, 1, 1, 9, 1, 9, 1, 9, 1, 1, 9, 9, 9,
       9, 1, 1, 1, 1, 9, 1, 9, 1, 1])>

In [45]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([6, 8, 3, 5, 8, 1, 9, 6, 4, 2, 0, 3, 1, 9, 0, 3, 6, 5, 4, 6, 6, 4,
       8, 9, 3, 6, 4, 1, 4, 7, 0, 7])>

In [46]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

In [47]:
train_loss = tf.keras.metrics.Mean('train_loss')#meteics计算损失对象
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
#计算每一步的训练准确率

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [48]:
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss_step)#返回loss的均值
    train_accuracy(labels, pred)#返回准确率均值

In [49]:
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [50]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        print('Epoch{} loss is {}, accuracy is {}'.format(epoch,
                                                          train_loss.result(),
                                                          train_accuracy.result()))
        
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
        print('Epoch {} test_loss is {}, test_accuracy is {}'.format(epoch,
                                                              test_loss.result(),
                                                              test_accuracy.result()))
        
        train_loss.reset_states()#重新设置训练参数的状态
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()

In [51]:
train()

Epoch0 loss is 0.9328579902648926, accuracy is 0.7062000036239624
Epoch 0 test_loss is 0.4415159821510315, test_accuracy is 0.8587999939918518
Epoch1 loss is 0.3824250400066376, accuracy is 0.8821166753768921
Epoch 1 test_loss is 0.3102761209011078, test_accuracy is 0.9038000106811523
Epoch2 loss is 0.3067557215690613, accuracy is 0.90461665391922
Epoch 2 test_loss is 0.28605541586875916, test_accuracy is 0.9100000262260437
Epoch3 loss is 0.27012768387794495, accuracy is 0.914900004863739
Epoch 3 test_loss is 0.2884828746318817, test_accuracy is 0.9072999954223633
Epoch4 loss is 0.24741436541080475, accuracy is 0.9226499795913696
Epoch 4 test_loss is 0.2165815681219101, test_accuracy is 0.9311000108718872
Epoch5 loss is 0.22932228446006775, accuracy is 0.9286666512489319
Epoch 5 test_loss is 0.2244223952293396, test_accuracy is 0.9275000095367432
Epoch6 loss is 0.2168780267238617, accuracy is 0.9328500032424927
Epoch 6 test_loss is 0.20191755890846252, test_accuracy is 0.93430000543594

### tf.keras.metrics 汇总计算模块

In [52]:
model.save('useeager.h5')

In [53]:
m = tf.keras.metrics.Mean('acc')

In [54]:
m(10)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [55]:
m(20)

<tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [56]:
m.result().numpy()

15.0

In [57]:
m([30, 40])

<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

In [58]:
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

In [59]:
m.reset_states()

In [60]:
m(1)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [61]:
m(2)

<tf.Tensor: shape=(), dtype=float32, numpy=1.5>

In [62]:
m.result().numpy()

1.5

In [63]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')

In [64]:
a(labels, model(features))

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [65]:
1/32

0.03125

In [66]:
#train()

In [68]:
import tensorflow as tf
from tensorflow import keras
(train_image,train_label),(test_image,test_label)=keras.datasets.mnist.load_data()#导入数据
train_images=tf.expand_dims(train_image,-1)#扩展维度
train_images=tf.cast(train_images/255,tf.float32)#归一化处理数据，然后转化数据格式
train_label=tf.cast(train_label,tf.int64)#处理标签数据
train_data=tf.data.Dataset.from_tensor_slices((train_images,train_label))#制作数据
BATCH_SIZE=32
train_data=train_data.shuffle(10000).batch(BATCH_SIZE)
model=tf.keras.Sequential()
model.add(keras.layers.Conv2D(16,(3,3),input_shape=(28,28,1),activation='relu'))
model.add(keras.layers.Conv2D(32,(3,3),activation='relu'))
model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Dense(10))
#创建模型
optimizer=keras.optimizers.Adam()
loss_func=keras.losses.SparseCategoricalCrossentropy(from_logits = True)#大写的是可以直接调用的方法
def loss(model,x,y):
    y_=model(x)
    return loss_func(y,y_)
def train_step(model,image,label):
    with tf.GradientTape() as t:
        loss_1=loss(model,image,label)
    grad=t.gradient(loss_1,model.trainable_variables)
    optimizer.apply_gradients(zip(grad,model.trainable_variables))
def train():
    for epoch in range(5):
        for(batch,(images,labels)) in enumerate(train_data):
            train_step(model,images,labels)
        print("epoch  {}  is finished".format(epoch))


In [69]:
train()

epoch1874 is finished
epoch1874 is finished
epoch1874 is finished
epoch1874 is finished
epoch1874 is finished


In [70]:
features,label=next(iter(train_data))
features.shape#每一批次的数据为三十二个
prediction=model(features)
prediction.shape
tf.argmax(prediction, axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([5, 9, 3, 0, 8, 9, 8, 0, 2, 5, 9, 4, 9, 4, 8, 1, 0, 8, 6, 6, 2, 7,
       8, 3, 4, 9, 5, 5, 5, 1, 4, 4])>

In [ ]:
label